In [ ]:
import os
import sys
from pathlib import Path

import cv2
import numpy as np
import torch
from matplotlib import pyplot as plt
from monai.utils import set_determinism
from torch.utils.data import DataLoader

sys.path.insert(0, "..")

In [ ]:
from src.utils import setup_dirs

root_dir = Path(os.getcwd()).parent
data_dir, log_dir, out_dir = setup_dirs(root_dir)
data_dir = data_dir / "ACDC" / "database"

set_determinism(seed=42)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from src.transforms.nnunet_transforms import get_nnunet_transforms
from src.datasets.acdc_dataset import ACDCDataset
from src.sam.sam_utils import get_sam_points

train_transforms, val_transforms = get_nnunet_transforms()
train_data = ACDCDataset(data_dir=data_dir / "training", transform=train_transforms, random_slice=True)
test_data = ACDCDataset(
    data_dir=data_dir / "testing", transform=val_transforms, random_slice=True
)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True, num_workers=0)

In [ ]:
from segment_anything import sam_model_registry

model_type = "vit_h"
checkpoint = root_dir / "models" / "sam_vit_h_4b8939.pth"
sam = sam_model_registry[model_type](checkpoint=checkpoint)
sam = sam.to(device)

num_classes = 4
pos_sample_points = 3
neg_sample_points = 1


In [ ]:

def get_bounding_box(ground_truth_map: np.ndarray, bbox_shift=5) -> np.ndarray:
    # get bounding box from mask
    y_indices, x_indices = np.where(ground_truth_map > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = ground_truth_map.shape
    x_min = max(0, x_min - bbox_shift)
    x_max = min(W, x_max + bbox_shift)
    y_min = max(0, y_min - bbox_shift)
    y_max = min(H, y_max + bbox_shift)
    bboxes = np.array([x_min, y_min, x_max, y_max])

    return bboxes


def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels == 1]
    neg_points = coords[labels == 0]
    ax.scatter(
        pos_points[:, 0], pos_points[:, 1], color="green", marker="*", s=marker_size, edgecolor="white", linewidth=1.25
    )
    ax.scatter(
        neg_points[:, 0], neg_points[:, 1], color="red", marker="*", s=marker_size, edgecolor="white", linewidth=1.25
    )


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor="green", facecolor=(0, 0, 0, 0), lw=2))  # %%


def show_figure(num_classes: int, inputs, labels, masks, bboxes, points, point_labels):
    plt.figure(figsize=(10, 10))

    for i in range(num_classes):
        plt.subplot(num_classes, 3, i * 3 + 1)
        plt.imshow(inputs, cmap="gray")

        if bboxes[i] is not None:
            show_box(bboxes[i], plt.gca())

        if points[i] is not None:
            show_points(points[i], point_labels[i], plt.gca(), marker_size=100)

        plt.axis("off")
        plt.subplot(num_classes, 3, i * 3 + 2)
        plt.imshow(labels[i])
        plt.axis("off")
        plt.subplot(num_classes, 3, i * 3 + 3)
        show_mask(masks[i], plt.gca())
        plt.axis("off")

    plt.show()


In [ ]:

from segment_anything import SamPredictor

predictor = SamPredictor(sam)

batch = next(iter(test_loader))
inputs, labels = batch["image"][0].to(device), batch["label"][0].to(device, dtype=torch.uint8)

# The input to the SAM predictor needs to be HWC, where C = 3 in either RGB or BGR format
inputs = cv2.cvtColor(inputs.permute(1, 2, 0).numpy(), cv2.COLOR_GRAY2RGB)
# Scale to 0-255, convert to uint8
inputs = ((inputs - inputs.min()) * (1 / (inputs.max() - inputs.min()) * 255)).astype("uint8")
predictor.set_image(inputs)

masks = []
labels = labels.cpu().numpy()

bboxes = []
points, point_labels = (
    get_sam_points(labels, num_classes, 5, neg_sample_points)
)
# Get bounding box for each class of one-hot encoded mask
for class_index in range(num_classes):
    bbox = get_bounding_box(labels[class_index])
    # bbox = None

    ground_truth_map, _, _ = predictor.predict(multimask_output=False, box=bbox, point_coords=points[class_index],
                                               point_labels=point_labels[class_index])
    masks.append(ground_truth_map)
    bboxes.append(bbox)


In [ ]:
masks[0].shape

In [ ]:
show_figure(num_classes, inputs, labels, masks, bboxes, points, point_labels)

In [ ]:
eps = 1e-6
dice_scores = []
for class_index in range(num_classes):
    # Ignore background class
    if class_index == 0: continue

    ground_truth = (labels == class_index).astype(int)

    tp = masks[class_index] * ground_truth
    fp = masks[class_index] * (1 - ground_truth)
    fn = (1 - masks[class_index]) * ground_truth
    tn = (1 - masks[class_index]) * (1 - ground_truth)

    dice = (2 * tp.sum() + eps) / (2 * tp.sum() + fp.sum() + fn.sum() + eps)
    print(f"Class {class_index} dice: {dice:.3f}")
    dice_scores.append(dice)

print(f"Average dice: {np.mean(dice_scores):.3f}")

In [ ]:
def prepare_image(image, transform, device):
    image = cv2.cvtColor(image.permute(1, 2, 0).numpy(), cv2.COLOR_GRAY2RGB)
    image = ((image - image.min()) * (1 / (image.max() - image.min()) * 255)).astype("uint8")
    image = transform.apply_image(image)
    image = torch.as_tensor(image, device=device)
    return image.permute(2, 0, 1).contiguous()

In [ ]:
def calculate_dice_per_class(masks, labels, ignore_background=True):
    dice_scores = []
    for class_index, mask in enumerate(masks):
        if ignore_background and class_index == 0: continue

        ground_truth = (labels == class_index).astype(int)

        tp = mask * ground_truth
        fp = mask * (1 - ground_truth)
        fn = (1 - mask) * ground_truth
        # tn = (1 - mask) * (1 - ground_truth)

        dice = (2 * tp.sum() + eps) / (2 * tp.sum() + fp.sum() + fn.sum() + eps)
        # print(f"Class {class_index} dice: {dice:.3f}")
        dice_scores.append(dice)

    return dice_scores


In [ ]:
from segment_anything.utils.transforms import ResizeLongestSide
from src.sam.sam_utils import get_batch_predictions

resize_transform = ResizeLongestSide(sam.image_encoder.img_size)

for batch in test_loader:
    inputs, labels, patient = (
        batch["image"].to(device),
        batch["label"].to(device, dtype=torch.uint8),
        batch["patient"],
    )

    with torch.no_grad():
        masks, boxes, points, point_labels = get_batch_predictions(
            sam=sam,
            transform=resize_transform,
            inputs=inputs,
            labels=labels,
            patients=patient,
            pos_sample_points=pos_sample_points,
            neg_sample_points=neg_sample_points,
            num_classes=num_classes,
            use_bboxes=True
        )
        masks = [mask.cpu().numpy() for mask in masks]

    break


In [ ]:
print(masks[0].shape)

In [ ]:
show_figure(num_classes, inputs[0].permute(1, 2, 0), labels[0], masks[0], boxes[0], points[0], point_labels[0])